# ◆[Backtest](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy) with OANDA API
###### Create Date：2020/02/08　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【table of contents】
- 0.[**Import module**](#Import_module)
- 1.[**Import data**](#Import_data)
- 2.[**Strategy**](#Strategy)
- 3.[**Backtest**](#Backtest)

- reference:https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html

## 0. Import module<a id='Import_module'></a>
- pip install backtesting
- pip install mpl_finance
- pip install oandapyV20
- pip install git+https://github.com/oanda/oandapy.git

In [1]:
%matplotlib inline
import pandas as pd
import oandapyV20.endpoints.instruments as instruments
import oandapy
import numpy   as np
import matplotlib.dates as mdates
import configparser
from datetime import datetime
from oandapyV20 import API
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

config = configparser.ConfigParser()
config.read('./config_OANDA.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

import warnings
warnings.filterwarnings("ignore")

def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

# ============================================================================
# get_rci
# ============================================================================
def RCI(close, period):
    rank_period = np.arange(period, 0, -1)
    length = len(close)
    rci = np.zeros(length)
    
    for i in range(length):
        if i < period - 1:
            rci[i] = 0
        else :
            rank_price = pd.Series(close)[i - period + 1: i + 1].rank(method='min', ascending = False).values
            rci[i] = np.int32((1 - (6 * sum((rank_period - rank_price)**2)) / (period**3 - period)) * 100)
            
    return rci


C:\Users\arsma\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 1. Import data<a id='Import_data'></a>

URL:http://developer.oanda.com/rest-live/rates/

In [2]:
oa = oandapy.API(environment="live", access_token=api_key)
data = oa.get_history(instrument="USD_JPY", granularity="D", count=1000)
 
# 取得したデータをpandasのDataFrameに変換
df = pd.DataFrame(data["candles"])
df["time"] = pd.to_datetime(df["time"])

display(df.tail(3))

df["time"] = df["time"].apply(mdates.date2num)

# backtesting.pyに必要なデータだけ取り出す
df_ohlc = df[["time","openAsk","highAsk","lowAsk","closeAsk"]].copy()
# backtesting.pyのデータに合わせてcolumns名を変更
df_ohlc = df_ohlc.rename(columns={"openAsk":"Open","highAsk":"High","lowAsk":"Low","closeAsk":"Close"})

# インデックスを変更する
df_ohlc = df_ohlc.set_index("time")

print('df_ohlc.shape=',df_ohlc.shape)
display(df_ohlc.tail(3))


,closeAsk,closeBid,complete,highAsk,highBid,lowAsk,lowBid,openAsk,openBid,time,volume
997,109.863,109.789,True,109.863,109.841,109.310,109.298,109.533,109.508,2020-02-04 22:00:00+00:00,77946
998,110.004,109.987,True,110.010,109.998,109.784,109.693,109.857,109.805,2020-02-05 22:00:00+00:00,55754
999,109.799,109.699,True,110.049,110.012,109.538,109.527,109.998,109.978,2020-02-06 22:00:00+00:00,69115


df_ohlc.shape= (1000, 4)


,Open,High,Low,Close
time,,,,
737459.916667,109.533,109.863,109.310,109.863
737460.916667,109.857,110.010,109.784,110.004
737461.916667,109.998,110.049,109.538,109.799


# 2. Strategy<a id='Strategy'></a>

In [3]:
# 売買戦略
class SmaCross(Strategy):
    
    n1 = 25 # 終値のSMA（単純移動平均）の期間
    n2 = 75 # 終値のSMA（単純移動平均）の期間
    
    d_rsi = 14  # Daily RSI lookback periods

    rci_s = 9
    rci_m = 21
    rci_l = 52

    def init(self): # ストラテジーの事前処理
        self.sma1 = self.I(SMA, self.data.Close, self.n1) # 終値のSMA（単純移動平均）をインジケータとして追加
        self.sma2 = self.I(SMA, self.data.Close, self.n2) # 終値のSMA（単純移動平均）をインジケータとして追加
 
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
    
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        
    def next(self): # ヒストリカルデータの行ごとに呼び出される（データの2行目から開始
        
        price = self.data.Close[-1]
        
        # *******************************************************************************************
        # buy
        # *******************************************************************************************
        # まだポジションがなく、すべての条件が満たされている場合は、longを入力します。
        #if crossover(self.sma1, self.sma2): # 
        if (not self.position and self.rci_s[-1] < -70 
            and self.rci_m[-1] < -50 
            and self.rci_l[-1] < -50
           ):
            
            #  次回のオープン時に市場価格で購入しますが、8％の固定ストップロスを設定します。
            # self.buy() # 現在のポジションを閉じて、所持金分買う
            self.buy(sl=.92 * price)
            
        # *******************************************************************************************
        # sell
        # *******************************************************************************************
        # 価格が10日移動平均より2％以上下がった場合、ポジションをクローズします
        #elif crossover(self.sma2, self.sma1):
        elif (self.rci_m[-1] > 90):
            
            #self.sell() # 現在のポジションを閉じて、所持金分売る
            self.position.close()
            
            
            

# 3. Backtest<a id='Backtest'></a>

- URL:https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy
- cash引数はバックテストで使う原資の金額をドル単位で入力します。今回は500ドルとしました。commissionは取引所の手数料を指定することが可能です。今回は0です。

In [4]:
# バックテストの実行
bt = Backtest(df_ohlc, # データ
              SmaCross,  # 戦略
              cash=1000,  # 所持金(ドル)
              commission=0.002, # 取引手数料（為替価格に対する倍率で指定、為替価格100円でcommission=0.0005なら0.05円）
              trade_on_close=True # True：現在の終値で取引する、False：次の時間の始値で取引する
             )
bt.run()

Start                       736067
End                         737462
Duration                   1395.04
Exposure [%]               29.3898
Equity Final [$]           1265.25
Equity Peak [$]            1273.02
Return [%]                 26.5251
Buy & Hold Return [%]     0.408772
Max. Drawdown [%]         -3.52559
Avg. Drawdown [%]        -0.753274
Max. Drawdown Duration     183.042
Avg. Drawdown Duration     25.6667
# Trades                         5
Win Rate [%]                   100
Best Trade [%]               12.76
Worst Trade [%]           0.628124
Avg. Trade [%]             4.90062
Max. Trade Duration        154.958
Avg. Trade Duration             82
Expectancy [%]                 NaN
SQN                        2.58066
Sharpe Ratio               1.03848
Sortino Ratio                  NaN
Calmar Ratio               1.39001
_strategy                 SmaCross
dtype: object

- Start        ヒストリカルデータの開始日時
- End          ヒストリカルデータの終了日時
- Duration     ヒストリカルデータの期間
- Exposure (%) ポジションを持っていた期間の割合（ポジションを持っていた期間÷全期間×100）
- Equity Final 最終金額
- Equity Peak  最高金額
- Return (%)   利益率=損益÷開始時所持金×100
- Buy & Hold Return (%)                0.101896  （（終了時の終値 - 開始時の終値）÷ 開始時の終値）の絶対値×100
- Max. Drawdown (%)                    -2.94051  最大下落率
- Avg. Drawdown (%)                  -0.0155023  平均下落率
- Max. Drawdown Duration        0 days 00:30:00  最大下落期間
- Avg. Drawdown Duration        0 days 00:10:00  平均下落期間
- Trades                                   54  取引回数
- Win Rate (%)                          9.25926  勝率=勝ち取引回数÷全取引回数×100
- Best Trade (%)                      0.0639561  1回の取引での利益の最大値÷所持金×100
- Worst Trade (%)                     -0.149003  1回の取引での損失の最大値÷所持金×100
- Avg. Trade (%)                     -0.0529873  損益の平均値÷所持金×100
- Max. Trade Duration           0 days 01:50:00  1回の取引での最長期間
- Avg. Trade Duration           0 days 00:25:00  1回の取引での平均期間
- Expectancy (%)                      0.0566947  期待値=平均利益×勝率＋平均損失×敗率
- 期待値とは、１回の取引で期待できる利益のことです。
- 売買ルールの優劣を判断する指標で、勝率、平均利益、平均損失から求めます。
- 期待値が正の場合は資産が増えていき、負の場合は資産が減っていくと考えます。

SQN                                  -10.6372 - SQN（SystemQualityNumber）
- SQNとは、Van K. Tharp氏によって定義された取引システムの分類（評価）方法です。
- SQN値によって下記のように分類されます。
- 1.6-1.9：平均以下
- 2.0-2.4：平均
- 2.5-2.9：良い
- 3.0-5.0：素晴らしい
- 5.1-6.9：最高
- 7.0～：聖杯？
- 取引数が30以上の場合、SQN値は信頼できるとみなされるでしょう。

Sharpe Ratio                         -1.45463 - 標準偏差に対するリターンの比率
- シャープレシオとは利益とリスクの比率のことで、値が大きいほど資産曲線がなめらかになり安定性のある利益が見込めます。

Sortino Ratio                        -1.87251 - 下方リスクに対するリターンの比率
- シャープレシオだけでは分からない下方リスクの抑制度合いを判断する場合に使われます。
- 通常、この数値が大きいほど優れている（下落局面に強い）ことを示します。

Calmar Ratio                       -0.0180198 - 最大損失率に対する年間平均収益の比率
- 値が低いほど指定された期間に渡ってリスク調整ベースで実行された投資は悪化します。
- 値が高いほどパフォーマンスが向上します。

_strategy                          myStrategy - 使用したストラテジーの関数名

In [5]:
bt.plot()
 